In [139]:
# import the necessary Libraries 
import sys
import requests
import json

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors


import io
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

In [140]:
#We get the information of postal codes for the major districts in Tokyo from the following website

url = "http://japanzipcodes.blogspot.com/2013/08/the-major-zip-codes-of-tokyo-japan.html"

url_u = requests.get(url).text

soup = BeautifulSoup(url_u, 'html.parser')

#We need to extract the information and store it in two lists: "district" and "postal_code" 

zips = soup.find(id="post-body-1523308068074273552")


tokyo_list = list(zips)



district = []
postal_code = []


for i in range(0,108,2):
    index = tokyo_list[i]
    if i not in (74,76,78):
        district_index = index.split(',')[0].split('      ')[1].split(' ')[0]
        postal_code_index = index.split(',')[2].split('is ')[1].split(' ')[0]    
        #append items
        district.append(district_index)
        postal_code.append(postal_code_index)    
        #three items have different parsing
    else: 
        district_index = index.split(',')[0].split('    ')[1].split(' ')[0]
        postal_code_index = index.split(',')[3].split('is ')[1].split(' ')[0]
        #append items
        district.append(district_index)
        postal_code.append(postal_code_index)

In [141]:
#We build a dataframe including the districts and the corresponding postal_codes
tokyo = {'District':district,'Postal_Code':postal_code}
tokyo_df = pd.DataFrame(tokyo)
#Adachiku is better known as Adachi. So we change that to not get a problem later in geolocating
#Same for a few other
tokyo_df['District'][0] = "Adachi"
tokyo_df['District'][4] = "Bunkyo"
tokyo_df['District'][5] = "Chiyoda"
tokyo_df['District'][18] = "Itabashi"
tokyo_df['District'][19] = "Katsushika"
tokyo_df['District'][36] = "Niijima"
tokyo_df['District'][39] = "Okutama"
tokyo_df['District'][43] = "Oshima"
tokyo_df['District'][46] = "Shibuya"
tokyo_df['District'][47] = "Shinagawa"
tokyo_df['District'][48] = "Shinjuku"
#I noticed during the assessment that I do not explicitly need the postal codes but I leave them in nevertheless :D

In [142]:
tokyo_df

,District,Postal_Code
0,Adachi,120-0000
1,Akiruno,190-0100
2,Akishima,196-0000
3,Arakawaku,116-0000
4,Bunkyo,112-0000
5,Chiyoda,100-0000
6,Chofu,182-0000
7,Chuoku,103-0000
8,Edogawaku,132-0000
9,Fuchu,183-0000


In [143]:
districts = tokyo_df['District']

lat = []
lon = []

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent")
city ="Tokyo"
country ="Japan"

for i in districts:
    print(i)
    loc = geolocator.geocode(i +','+"Tokyo"+','+ "Japan")
    lat.append(loc.latitude)
    lon.append(loc.longitude)


Adachi
Akiruno
Akishima
Arakawaku
Bunkyo
Chiyoda
Chofu
Chuoku
Edogawaku
Fuchu
Fussa
Hachioji
Hamura
Higashikurume
Higashimurayama
Higashiyamato
Hino
Inagi
Itabashi
Katsushika
Kitaku
Kiyose
Kodaira
Koganei
Kokubunji
Komae
Kotoku
Kunitachi
Machida
Meguroku
Minatoku
Mitaka
Musashimurayama
Musashino
Nakanoku
Nerimaku
Niijima
Hinodemachi
Hinoharamura
Okutama
Nishitokyo
Ogasawaramura
Ome
Oshima
Otaku
Setagayaku
Shibuya
Shinagawa
Shinjuku
Suginamiku
Sumidaku
Tachikawa
Taitoku
Tama


In [145]:
#add the coordinates to the existing data frame
tokyo_df['Latitude'] = lat
tokyo_df['Longitude'] = lon

In [146]:
tokyo_df

,District,Postal_Code,Latitude,Longitude
0,Adachi,120-0000,35.783703,139.795319
1,Akiruno,190-0100,35.731042,139.217028
2,Akishima,196-0000,35.702480,139.350065
3,Arakawaku,116-0000,35.737529,139.781310
4,Bunkyo,112-0000,35.718810,139.744732
5,Chiyoda,100-0000,35.685403,139.752738
6,Chofu,182-0000,35.660036,139.554815
7,Chuoku,103-0000,34.545288,135.840885
8,Edogawaku,132-0000,35.706653,139.868312
9,Fuchu,183-0000,35.672219,139.480009


In [147]:
#coordinates of Tokyo

address = 'Tokyo, Japan'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tokyo are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tokyo are 35.6828387, 139.7594549.


In [149]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [150]:
# create map of Tokyo
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers off the districts
for lat, lng, label in zip(tokyo_df['Latitude'], tokyo_df['Longitude'], tokyo_df['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [151]:
CLIENT_ID = '2ZX4HJMJUV2EXPLKE3JGJPEEJGZ2WSBQA4UL0DHI1ZWIEX2U' # your Foursquare ID
CLIENT_SECRET = 'EY145003SSWRH5H321SPTJEVX33D2SRUXK2M01Y331FCTTJX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2ZX4HJMJUV2EXPLKE3JGJPEEJGZ2WSBQA4UL0DHI1ZWIEX2U
CLIENT_SECRET:EY145003SSWRH5H321SPTJEVX33D2SRUXK2M01Y331FCTTJX


In [157]:
#we use the function getNearbyVenues from the tutorial and modify it so that we get venues that are found via "video games"
#therefore we include the query 'video games'
#radius = 500 seems like a good choice, so we leave the standard value from the tutorial

def getNearbyVenues(names, latitudes, longitudes, radius=500, query = 'video games'):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            query)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [158]:
tokyo_vgs = getNearbyVenues(names=tokyo_df['District'],
                                   latitudes=tokyo_df['Latitude'],
                                   longitudes=tokyo_df['Longitude']
                                  )
#tokyo_vgs only contains venues that include "video games". Nevertheless, there may be different venue categories
#since video games may be sold or playable or so in venues like a cafe, an arcade, a restaurant, even bus stop
#->arcade machines for waiting for the bus, etc...

Adachi
Akiruno
Akishima
Arakawaku
Bunkyo
Chiyoda
Chofu
Chuoku
Edogawaku
Fuchu
Fussa
Hachioji
Hamura
Higashikurume
Higashimurayama
Higashiyamato
Hino
Inagi
Itabashi
Katsushika
Kitaku
Kiyose
Kodaira
Koganei
Kokubunji
Komae
Kotoku
Kunitachi
Machida
Meguroku
Minatoku
Mitaka
Musashimurayama
Musashino
Nakanoku
Nerimaku
Niijima
Hinodemachi
Hinoharamura
Okutama
Nishitokyo
Ogasawaramura
Ome
Oshima
Otaku
Setagayaku
Shibuya
Shinagawa
Shinjuku
Suginamiku
Sumidaku
Tachikawa
Taitoku
Tama


In [160]:
tokyo_vgs

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adachi,35.783703,139.795319,7-Eleven (セブンイレブン 足立島根店),35.782408,139.797570,Convenience Store
1,Adachi,35.783703,139.795319,ウエルシア薬局 足立島根店,35.780319,139.797193,Drugstore
2,Adachi,35.783703,139.795319,FamilyMart (ファミリーマート 島根二丁目店),35.782771,139.799411,Convenience Store
3,Adachi,35.783703,139.795319,COCO'S (ココス),35.782871,139.797551,Japanese Family Restaurant
4,Adachi,35.783703,139.795319,FamilyMart (ファミリーマート 島根四丁目店),35.782487,139.792864,Convenience Store
...,...,...,...,...,...,...,...
1301,Tama,35.637188,139.443503,Dining和桜,35.640556,139.444600,Japanese Restaurant
1302,Tama,35.637188,139.443503,諏訪下橋バス停,35.636195,139.447745,Bus Stop
1303,Tama,35.637188,139.443503,Sakuragaoka Rotary (桜ヶ丘ロータリー (地球屋前ロータリー)),35.640814,139.444469,Road
1304,Tama,35.637188,139.443503,原峰公園,35.640998,139.445108,Park


In [170]:
#as in the manhattan tutorial we check how many venues were returned for each district
tokyo_vgs.groupby('District').count()[['Venue']]

,Venue
District,
Adachi,11
Akiruno,4
Akishima,16
Arakawaku,36
Bunkyo,15
Chiyoda,17
Chofu,8
Chuoku,5
Edogawaku,29


In [175]:
#we now check how many unique venue categories (only "video games" related)
#there are
print('There are {} uniques categories.'.format(len(tokyo_vgs['Venue Category'].unique())))

There are 208 uniques categories.


In [176]:
# one hot encoding
tokyo_vgs_onehot = pd.get_dummies(tokyo_vgs[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tokyo_vgs_onehot['District'] = tokyo_vgs['District'] 

# move neighborhood column to the first column
fixed_columns = [tokyo_vgs_onehot.columns[-1]] + list(tokyo_vgs_onehot.columns[:-1])
tokyo_vgs_onehot = tokyo_vgs_onehot[fixed_columns]

tokyo_vgs_onehot.head()

,District,ATM,Accessories Store,African Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,...,Unagi Restaurant,Used Bookstore,Video Store,Vietnamese Restaurant,Wagashi Place,Waterfall,Wine Bar,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant
0,Adachi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Adachi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Adachi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Adachi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adachi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [177]:
#check data size as in manhattan example
tokyo_vgs_onehot.shape

(1306, 209)

In [179]:
#group rows by district and by taking the mean of the frequency of occurrence of each category
tokyo_vgs_grouped = tokyo_vgs_onehot.groupby('District').mean().reset_index()
tokyo_vgs_grouped

,District,ATM,Accessories Store,African Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,...,Unagi Restaurant,Used Bookstore,Video Store,Vietnamese Restaurant,Wagashi Place,Waterfall,Wine Bar,Wings Joint,Yakitori Restaurant,Yoshoku Restaurant
0,Adachi,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Akiruno,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25
2,Akishima,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Arakawaku,0.000000,0.00,0.00,0.000000,0.000000,0.027778,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Bunkyo,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Chiyoda,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Chofu,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Chuoku,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.00
8,Edogawaku,0.000000,0.00,0.00,0.000000,0.000000,0.034483,0.000000,0.000000,0.0000,...,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Fuchu,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00


In [180]:
tokyo_vgs_grouped.shape

(53, 209)

In [181]:
#each district with top 10 most common venues (only venues connecte to "video games")
num_top_venues = 10

for hood in tokyo_vgs_grouped['District']:
    print("----"+hood+"----")
    temp = tokyo_vgs_grouped[tokyo_vgs_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adachi----
                        venue  freq
0           Convenience Store  0.27
1         Japanese Restaurant  0.18
2                    Sake Bar  0.09
3  Japanese Family Restaurant  0.09
4                   Drugstore  0.09
5                 Supermarket  0.09
6                        Park  0.09
7                    Pharmacy  0.09
8                  Playground  0.00
9       Outdoors & Recreation  0.00


----Akiruno----
                   venue  freq
0     Yoshoku Restaurant  0.25
1          Grocery Store  0.25
2      Convenience Store  0.25
3         History Museum  0.25
4             Playground  0.00
5  Outdoors & Recreation  0.00
6                 Palace  0.00
7                   Park  0.00
8            Pastry Shop  0.00
9               Pet Café  0.00


----Akishima----
                    venue  freq
0       Convenience Store  0.19
1            Intersection  0.19
2               BBQ Joint  0.06
3             Supermarket  0.06
4  Furniture / Home Store  0.06
5          Clothing

9             Supermarket  0.06


----Musashimurayama----
               venue  freq
0  Convenience Store  0.08
1       Intersection  0.08
2      Grocery Store  0.08
3         Bike Trail  0.08
4       Concert Hall  0.08
5    Motorcycle Shop  0.08
6             Tunnel  0.08
7    Udon Restaurant  0.08
8           Bus Stop  0.08
9        Karaoke Box  0.08


----Musashino----
                  venue  freq
0     Convenience Store  0.33
1    Chinese Restaurant  0.13
2      Sushi Restaurant  0.07
3              Bus Stop  0.07
4   Japanese Restaurant  0.07
5           Supermarket  0.07
6           Pastry Shop  0.07
7       Cultural Center  0.07
8  Fast Food Restaurant  0.07
9      Ramen Restaurant  0.07


----Nakanoku----
                venue  freq
0   Convenience Store  0.09
1           BBQ Joint  0.09
2    Ramen Restaurant  0.09
3  Italian Restaurant  0.09
4                Café  0.09
5            Sake Bar  0.09
6        Noodle House  0.06
7                Park  0.06
8       Grocery Store  0

In [182]:
#sort venues desending order  
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [184]:
#create dataframe
#we leave the term "neighbourhood".  could change it but does not matter for the code here. 
#the corresponding column is called 'district' and that is all that matters for our purposes
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['District'] = tokyo_vgs_grouped['District']

for ind in np.arange(tokyo_vgs_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tokyo_vgs_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Adachi,Convenience Store,Japanese Restaurant,Supermarket,Park,Pharmacy,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest
1,Akiruno,Yoshoku Restaurant,Convenience Store,Grocery Store,History Museum,Fast Food Restaurant,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest,Food & Drink Shop,Food,Flea Market,Fishing Store,Farmers Market
2,Akishima,Intersection,Convenience Store,Discount Store,BBQ Joint,Furniture / Home Store,Bus Stop,Park,Motorcycle Shop,Clothing Store,Shipping Store,Supermarket,Auto Workshop,Event Space,Drugstore,Donburi Restaurant
3,Arakawaku,Park,Grocery Store,Convenience Store,Bus Stop,Intersection,Noodle House,Ramen Restaurant,Chinese Restaurant,Concert Hall,Café,Donburi Restaurant,Theater,Historic Site,Bath House,Asian Restaurant
4,Bunkyo,Convenience Store,Japanese Restaurant,Botanical Garden,Grocery Store,Museum,Park,Road,Italian Restaurant,Chinese Restaurant,Café,Steakhouse,Szechuan Restaurant,Intersection,BBQ Joint,Bar


In [185]:
#we now cluster the districts into 3 clusters
# set number of clusters
kclusters = 3

tokyo_vgs_grouped_clustering = tokyo_vgs_grouped.drop('District', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tokyo_vgs_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [186]:
#set up new data frame containing cluster and top 15 venues
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tokyo_vgs_merged = tokyo_vgs

# merge tokyo_vgs_grouped with tokyo_vgs to add latitude/longitude for each district
tokyo_vgs_merged = tokyo_vgs_merged.join(neighborhoods_venues_sorted.set_index('District'), on='District')

tokyo_vgs_merged.head() # check the last columns!

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Adachi,35.783703,139.795319,7-Eleven (セブンイレブン 足立島根店),35.782408,139.797570,Convenience Store,0,Convenience Store,Japanese Restaurant,...,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest
1,Adachi,35.783703,139.795319,ウエルシア薬局 足立島根店,35.780319,139.797193,Drugstore,0,Convenience Store,Japanese Restaurant,...,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest
2,Adachi,35.783703,139.795319,FamilyMart (ファミリーマート 島根二丁目店),35.782771,139.799411,Convenience Store,0,Convenience Store,Japanese Restaurant,...,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest
3,Adachi,35.783703,139.795319,COCO'S (ココス),35.782871,139.797551,Japanese Family Restaurant,0,Convenience Store,Japanese Restaurant,...,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest
4,Adachi,35.783703,139.795319,FamilyMart (ファミリーマート 島根四丁目店),35.782487,139.792864,Convenience Store,0,Convenience Store,Japanese Restaurant,...,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest


In [188]:
#visualize clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tokyo_vgs_merged['District Latitude'], tokyo_vgs_merged['District Longitude'], tokyo_vgs_merged['District'], tokyo_vgs_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

,District Latitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,35.783703,7-Eleven (セブンイレブン 足立島根店),35.782408,139.797570,Convenience Store,0,Convenience Store,Japanese Restaurant,Supermarket,Park,...,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest
1,35.783703,ウエルシア薬局 足立島根店,35.780319,139.797193,Drugstore,0,Convenience Store,Japanese Restaurant,Supermarket,Park,...,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest
2,35.783703,FamilyMart (ファミリーマート 島根二丁目店),35.782771,139.799411,Convenience Store,0,Convenience Store,Japanese Restaurant,Supermarket,Park,...,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest
3,35.783703,COCO'S (ココス),35.782871,139.797551,Japanese Family Restaurant,0,Convenience Store,Japanese Restaurant,Supermarket,Park,...,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest
4,35.783703,FamilyMart (ファミリーマート 島根四丁目店),35.782487,139.792864,Convenience Store,0,Convenience Store,Japanese Restaurant,Supermarket,Park,...,Drugstore,Sake Bar,Japanese Family Restaurant,Auto Garage,Garden Center,Gaming Cafe,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,35.637188,Dining和桜,35.640556,139.444600,Japanese Restaurant,0,Bus Stop,Japanese Restaurant,Convenience Store,Intersection,...,Park,Italian Restaurant,Wings Joint,Playground,Chinese Restaurant,Candy Store,Sushi Restaurant,Hotel Bar,Forest,Food
1302,35.637188,諏訪下橋バス停,35.636195,139.447745,Bus Stop,0,Bus Stop,Japanese Restaurant,Convenience Store,Intersection,...,Park,Italian Restaurant,Wings Joint,Playground,Chinese Restaurant,Candy Store,Sushi Restaurant,Hotel Bar,Forest,Food
1303,35.637188,Sakuragaoka Rotary (桜ヶ丘ロータリー (地球屋前ロータリー)),35.640814,139.444469,Road,0,Bus Stop,Japanese Restaurant,Convenience Store,Intersection,...,Park,Italian Restaurant,Wings Joint,Playground,Chinese Restaurant,Candy Store,Sushi Restaurant,Hotel Bar,Forest,Food
1304,35.637188,原峰公園,35.640998,139.445108,Park,0,Bus Stop,Japanese Restaurant,Convenience Store,Intersection,...,Park,Italian Restaurant,Wings Joint,Playground,Chinese Restaurant,Candy Store,Sushi Restaurant,Hotel Bar,Forest,Food


,District Latitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
866,35.725567,華水の滝,35.729212,139.103003,Waterfall,1,Waterfall,Yoshoku Restaurant,Farmers Market,Gaming Cafe,...,Fried Chicken Joint,French Restaurant,Forest,Food & Drink Shop,Food,Flea Market,Fishing Store,Fast Food Restaurant,Farm,Garden Center


,District Latitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
912,35.803601,企業の森,35.805742,139.234461,Forest,2,Forest,Yoshoku Restaurant,Yakitori Restaurant,Garden,...,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food & Drink Shop,Food,Flea Market,Fishing Store,Fast Food Restaurant,Farmers Market,Farm
